In [6]:
import pandas as pd
import requests

NGROK_DOMAIN = "http://localhost:8080/"
session_id = 102292667


response = requests.post(f"{NGROK_DOMAIN}/get_paf_result", json={"session_id": session_id})
response = response.json()
table = pd.DataFrame.from_dict(response.get("responses", {}), orient='index')

In [10]:
table.reset_index(drop=True)[["session_id", "slot_name", "question_asked","slot_value"]]

,session_id,slot_name,question_asked,slot_value
0,102292667,cough_existence,有咳嘅。你會唔會咳痰呀？,yes
1,102292667,sputum_existence,"有痰呀,你咳嗽時候,係咪咳出好多痰,定係只係少少？",yes
2,102292667,sputum_amount,你見到嘅痰，係透明、黃色，定係有其他顏色？,the patient has experienced a lot of sputum.
3,102292667,sputum_color,三個月來，有冇試過失足或者摔跤呀？,the patient's sputum color is yellow.
4,102292667,history_of_falling,你行路時，需唔需要手揸住嘢幫手平衡？,yes
